Regular Expressions
-------------------

Regular expressions (regexes or re’s) constitute an extremely powerful, flexible and concise language for matching elements in text ranging from a few characters to complex patterns. While mastering the syntax of the regular expression language does require climbing a learning curve, this learning curve is not particularly steep, and a newcomer can find herself performing useful tasks with regular expressions almost immediately. Efforts spent learning regular expressions quickly pay off--tasks that are well suited for regular expressions abound. Indeed, regular expressions are one of the most useful computer skills, and an absolutely critical tool for data scientists. 

This document will present basic regular expression syntax and cover common use cases for regular expressions: pattern matching, filtering, data extraction, and string replacement. We will present examples using grep, which we covered previously. (In case you forgot, we used grep to find lines of a text file with a given string in them.) 

In [1]:
# The code below is written in Python to replicate the behavior of grep, the UNIX utility
# We will examine the details of how the code works in a subsequent notebook.
# For now, just execute the code, and use the function grep(regex_expression, file_name) as-is

import re

def printMatches(text, regex_expression):
    BACKGROUND_YELLOW = '\x1b[43m'
    COLOR_RESET  = "\x1b[0m"
    regex= re.compile(regex_expression)
    matches = regex.finditer(text)
    for m in matches:
        highlighted  = text[:m.start()] # the string before the regex match
        highlighted += BACKGROUND_YELLOW + text[m.start():m.end()] + COLOR_RESET 
        highlighted += text[m.end():] # the string after the regex match
        print(highlighted)

def grep(regex_expression, file_name):
    f = open(file_name, "r")
    content = f.read()
    f.close()
    for line in content.split("\n"):
        printMatches(line, regex_expression)

### NYC Restaurant Names Data

To have a longer data set to play with, let's download the list of restaurant names from the NYC Restaurant Inspection Dataset.

In [2]:
!cp /home/ubuntu/jupyter/NYU_Notes/2-Introduction_to_Python/data/restaurant-names.txt data/uniquenames.txt

Let's take a peek at the contents using the `head` and `tail` commands:

In [3]:
!head -10 data/uniquenames.txt
!echo '........' # The "echo" command just prints in the output the string that follows the command (in this case "......")
!tail -10 data/uniquenames.txt

#1 GARDEN CHINESE
#1 ME. NICK'S
#1 SABOR LATINO RESTAURANT
$1.25 PIZZA
''U'' LIKE CHINESE RESTAURANT
''W'' CAFE
'WICHCRAFT
(LEWIS DRUG STORE) LOCANDA VINI E OLII
(LIBRARY)  FOUR & TWENTY BLACKBIRDS
(PUBLIC FARE) 81ST STREET AND CENTRAL PARK WEST (DELACORTE THEATRE)
........
ZUCKER'S BAGELS AND SMOKED FISH
ZUM SCHNEIDER
ZUM STAMMTISCH
ZUMA JAPANESE RESTAURANT NEW YORK
ZUMBA RESTAURANT
ZUTTO
ZUZU RAMEN
ZYMI BAR & GRILL
ZZ CLAM BAR
ZZ'S PIZZA & GRILL


Now, let's see if there are any restaurants with the string 'PANO' in them:

In [4]:
grep('PANO', "data/uniquenames.txt")

BUFFALO WILD WINGS,PEETS COOFEE &TEA, PANOPOLIS BAKERY & CAFE
CAFE ESPANOL
EL CHARRO ESPANOL
EL POTE ESPANOL
LA CANDELA ESPANOLA
PAMPANO
PANORAMA OF MY SILENCE-HEART
PANORAMA RESTAURANT
TIGIN IRISH PUB,PEETS COFFEE&TEA,PANOPOLIS BAKERY&CAFE


What can we do if we want to search for something more complex than a fixed string? Regular expressions are solving exactly this problem. 

### The atoms

The simplest regular expressions are a sequence of `atoms`. An atom can be any of the following:
* single character, 
* a dot,
* a bracket expression, 
* an anchor.

#### Single character atom

A single character atom matches itself.

#### The `.` character atom

A dot atom matches any single character (except for a new line character `\n`).

Example: Using single character atoms, and the `.` atom, let's find all restaurant names that contain the characters `AB`, followed by any character (`.`) and then the character `D`:

In [5]:
grep('AB.D', 'data/uniquenames.txt')

ABIDE BROOKLYN PITA
JJ PEABODY'S
LABADEE MANOIR
NEW KABAB DINER
RESTAURANT ABIDJAN


In [6]:
grep('A.B.C', 'data/uniquenames.txt')

@ THE SQUARE(STARBUCKS)
ABBOCCATO RISTORANTE
D'ANGELO CENT/STARBUCKS COFFEE
DELANEY BARBECUE
HERALD SQUARE CAFE (STARBUCKS)
HILL COUNTRY BARBECUE MARKET
HUNTER COLLEGE PROUDLY BREW STARBUCKS
JAKE'S WAYBACK BURGERS
JOSE O SHEA'S/STARBUCKS
MALBEC TANGO HOUSE
MISS KOREA BARBECUE ''SUN''
NEELY'S BARBECUE PARLOR/MERCHANTS
PIZZA LA BOCA
PRATT DESIGN CENTER/STARBUCKS
RED MALBEC
SAMBUCA'S CAFE
SHILLA KOREAN BARBECUE HOUSE
STARBUCKS
STARBUCKS # 14840
STARBUCKS (JFK TERMINAL 5-POST SECURITY DEPARTURE)
STARBUCKS (STORE 16628)
STARBUCKS 22420
STARBUCKS COFFEE
STARBUCKS COFFEE  #16608
STARBUCKS COFFEE # 15440
STARBUCKS COFFEE # 7463
STARBUCKS COFFEE # 7540
STARBUCKS COFFEE #14240
STARBUCKS COFFEE #18509
STARBUCKS COFFEE #20679
STARBUCKS COFFEE #21514
STARBUCKS COFFEE #22596
STARBUCKS COFFEE #23266
STARBUCKS COFFEE #23267
STARBUCKS COFFEE #3438
STARBUCKS COFFEE #7344
STARBUCKS COFFEE #7358
STARBUCKS COFFEE #7416
STARBUCKS COFFEE #7682
STARBUCKS COFFEE #7826
STARBUCKS COFFEE #9282
STARBUCKS COFFEE #9

#### Bracket expression atom

A bracket expression (defined by square brackets []) defines a set of characters. matches only one single character that can be any of the characters defined in a set. Example: [ABL] matches either A, B, or L.

Now, let's use a bracket expression: We want to find restaurants that contain one of the letters A,B,C,X,Y,Z followed by a digit. We specify the set of letters as `[ABCXYZ]` and the set of digits as [0123456789].  

In [7]:
grep('[ABCXYZ][0123456789][0123456789]', 'data/uniquenames.txt')

B66 CLUB
BA1002 BAR
BA1019 BAR
BA6110 BAR
BC8140 BAR AT THE GARDEN
CB8030 SAUSAGE CONCESSION
CIBO MARKET (GATE C65)
COTTO MARKET-GATE C30
FA8070 HOT DOG CONCESSION
FB1014 HOT DOG CONCESSION
FB8020 PIZZA CONCESSION
FB9090 HOT DOG CONCESSION
FB9110 HOT DOG CONCESSION
FB9120 HOT DOG CONCESSION
HOT DOG CONCESSION A803-1
JFK FUEL BAR B27
MADISON CLUB (BB7184)
RUNWAY69
YOGURT Y23 INC


##### Brackets and ranges

Instead of typing long lists of characters in a bracket expression, we can use the range character: [0-9] is equivalent to [0123456789]. Similarly [A-Z] is equivalent to [ABCDEFGHIJKLMNOPQRSTUVWXYZ]. And [D-T] is equivalent to [DEFGHIJKLMNOPQRST]. (You get the idea.) You can also combine multiple ranges: [a-e1-9] is equivalent to [abcde123456789]. Finally, you can even specify to be excluded from the set using the character (^). For example, [^0-9] matches any character other than a number.

For example, let's find restaurants that contain a letter, followed by a number, and then followed by a charather that is not a number:

In [8]:
grep('[A-Z][0-9][^0-9]', 'data/uniquenames.txt')

A1 OCHA SUSHI
AH2 ICE TEA
B4 NYC
BT3 BAR
BT4 BAR
C2 CAFE
CAFE1 & CAFE 4 (AMERICAN MUSEUM OF NATURAL HISTORY)
F1 LOUNGE AND GRILL
ILLY/VELOCITY BAR (EC2)
J4 HOOKAH LOUNGE
JUICE4U
M1-5
M2M MART
M2N BUFFET
NINETY9 & UP DINER
NO1 CHINESE RESTAURANT
Q2 THAI RESTAURANT
T2 - GO
TERMINAL1 EMPLOYEE CAFETERIA
THE NEW YORK PALACE HOTEL (C1 LEVEL CAFETERIA)
TWO8TWO BAR & BURGER
US FRIED CHICKEN & P1ZZA


Hm, we do not want to get results that have a space after the number, so let's also exclude the space character:

In [9]:
grep('[A-Z][0-9][^0-9 ]', 'data/uniquenames.txt') 

ILLY/VELOCITY BAR (EC2)
JUICE4U
M1-5
M2M MART
M2N BUFFET
TWO8TWO BAR & BURGER
US FRIED CHICKEN & P1ZZA


In [10]:
# Digit, not letter not digit not space, digit
grep('[0-9][^A-Z0-9 ][0-9]', 'data/uniquenames.txt') 

$1.25 PIZZA
1.5 GALBI CORP
104-01 FOSTER AVENUE COFFEE SHOP(UPS)
36-02 DITMARS COFFEE CORP.
40/40 CLUB
40/40 CLUB BAR
44 1/2 CAFE
83 1/2
BRASSERIE 8 1/2
FOOD DEPOT 12*4
HOT DOG CONCESSION A803-1
M1-5
PRB 24-7
THE BEST $1.00 PIZZA


#### Anchor

Anchor atoms are used to define the location of a regex within a line. 

The anchor `^` specifies the *beginning of a line*, the anchor `$` specifies the end of a line. The anchor `\b` specifies the word boundary.

Example: Find restaurant names that start with the characters `BAL`

In [11]:
grep('^BAL', 'data/uniquenames.txt')

BALABOOSTA
BALADE
BALBOA RESTAURANT.
BALCON QUITENO RESTAURANT
BALDOR SPECIALTY FOODS
BALDUCCI'S
BALI NUSA INDONESIAN RESTAURANT
BALILO DELI
BALIMAYA RESTAURANT
BALKANIKA
BALKH SHISH KABAB HOUSE
BALL PARK HOT DOG
BALLARO
BALLATO'S RESTAURANT
BALLFIELDS CAFE
BALLI DELI & SALAD BAR
BALLY TOTAL FITNESS
BALLY'S SPORT CLUB
BALNDIE'S PLACE, INC
BALON
BALTHAZAR BAKERY
BALTHAZAR RESTAURANT
BALUCHI
BALUCHI'S
BALUCHI'S FRESH
BALUCHI'S INDIAN FOOD
BALVANERA
BALZEM


Example: Find restaurant names that end with the characters `NORTH`

In [12]:
grep('NORTH$', 'data/uniquenames.txt')

AQUEDUCT NORTH
BOURGEOIS PIG NORTH
PRATT INSTITUTE NORTH


Example: Let's try to find restaurants containing the word `COLUMBIA`:

In [13]:
grep('COLUMBIA', 'data/uniquenames.txt')

BROWNIE'S CAFE AT COLUMBIA
CAFE 212/COLUMBIA CATERING KITCHEN - ALFRED LERNER HALL
COLUMBIA UNIVERSITY MEDICAL CENTER BOOKSTORE CAFE
EL PUNTO COLUMBIANO RESTAURANTE BAKERY
LA GATA GOLOSA COLUMBIAN FOOD
PARAISO COLUMBIANO RESTAURANT
THE FACULTY CLUB (COLUMBIA UNIVERSITY)
THE SCHOOL AT COLUMBIA UNIVERSITY
TIERRAS COLUMBIANAS


Hm, something is wrong. We also get COLUMBIANO, COLUMBIANAS, and other words. We want only the word COLUMBIA, so we add the word anchors:

In [14]:
grep(r'\bCOLUMBIA\b', 'data/uniquenames.txt')

BROWNIE'S CAFE AT COLUMBIA
CAFE 212/COLUMBIA CATERING KITCHEN - ALFRED LERNER HALL
COLUMBIA UNIVERSITY MEDICAL CENTER BOOKSTORE CAFE
THE FACULTY CLUB (COLUMBIA UNIVERSITY)
THE SCHOOL AT COLUMBIA UNIVERSITY


#### In class exercises

Write a regular expression for:

* Match any character
* Match the end of line
* Match any digit
* Find all characters that are not digits
* Find all words with four letters
* Find every line that starts with a digit
* Find all empty lines
* Find all lines with 4 characters


### Regular Expressions: Operators

#### Alternation |

The alternation operator `|` defines one or more alternatives regular expressions that need to be true for the string to match the regular expression. 

For example, if we are looking for names that contain either the word `GREEK` or the word `RUSSIAN`, we issue the following command: 

In [15]:
grep('GREEK|RUSSIAN|FRENCH', 'data/uniquenames.txt')

ANTHI'S GREEK FOOD
AVLEE  GREEK KITCHEN
AVLEE GREEK KITCHEN
AVLI THE LITTLE GREEK TAVERN
BREEZE THAI-FRENCH KITCHEN
BY SUZETTE FRENCH CREPES
DIRTY FRENCH
ETHOS GREEK CUISINE
FRENCH CAFE GOURMAND
FRENCH DINER
FRENCH LOUIE
FRENCH ROAST
GREEK EXPRESS
GREEK FAMILY KITCHEN
GREEK GARDENS GRILL
GREEK GRILL
GREEK ISLANDS
GRK FRESH GREEK
GYRO GREEK STYLE
JEAN CLAUDE FRENCH BISTRO
JEAN DANET FRENCH PASTRY
JENNY FRENCH TOAST COFFEE SHOP RESTAURANT
MEDITERRANEAN GRILL GREEK TARVERNA
OKEANOS GREEK SEAFOOD
OPA! GREEK RESTAURANT
PIZZA AND FRENCH TASTE PIZZERIA
RAFINA GREEK CUISINE
RUSSIAN BATHS
RUSSIAN SAMOVAR
RUSSIAN TURKISH BATHS
SOMETHING GREEK
SYMPOSIUM GREEK RESTAURANT
THE GREEK
THE GREEK CORNER
THE GREEK KITCHEN  CLINTON BAKERY CAFE
THE RUSSIAN TEA ROOM
VILLAGE TAVERNA GREEK GRILL
ZENON TAVERNA GREEK RESTAURANT


The -E flag specifies that we will be using the "Extended Regular Expressions" standard (see the slides), which has more simplified syntax than the original regular expressions, which is the default for grep.

#### Repetition {m,n}

A repetition operator specifies that the atom or expression immediately before the repetition may be repeated. For example, if we are looking for restaurants that contain the letter I, three to five times:  

In [16]:
grep('I{3,5}', 'data/uniquenames.txt')

ANTIQUE CAFE & BAKERY III INC
AZOGUENITA BAKERY & RESTAURANT III
BAGEL EXPRESS III
BARZOLA'S RESTAURANT III
BREAD BROTHERS III
CESTRA'S PIZZA III
EL CHIVITO D'ORO III
EL PACHANGON III RESTAURANT & BAR
EL POLLO III
EL REY III
EMILIO III BAR
ESTRELLITA POBLANA III
GOLDEN DRAGON III
KNAPP PIZZA III
LAS NUEVAS EMPANADAS MONUMENTAL III
LITTLE ITALY PIZZA III
LOS POLLITOS III
MIRACALI III
NEW CHINA III
NEW FRESCO TORTILLOS III
NEW WIN HING III CHINESE RESTAURANT
ROCCO PIZZA III
SAKURA III
SHINJU III SUSHI
SUSHI TATSU JAPANESE RESTAURANT III
THI III NEW YORK
WARD III


Now, let's find all the restaurants that have a name length from 50 to 55 characters:

In [17]:
grep('^.{50,55}$', 'data/uniquenames.txt')

BRASSIERIE 1605/BROADWAY 49 BAR & LOUNGE (MAIN KITCHEN)
BROOKLYN CHILDREN'S MUSEUM CAFE/FOREST CITY RATNER CAFE
CAFE 212/COLUMBIA CATERING KITCHEN - ALFRED LERNER HALL
CAFE1 & CAFE 4 (AMERICAN MUSEUM OF NATURAL HISTORY)
CARIBBEAN CONNECTION CATERING SERVICES INC RESTAURANT
CHARTWELLS AT COLLEGE OF MOUNT ST. VINCENT-BENEDICT
COURTYARD & RESIDENCE INN BY MARRIOTT CENTRAL PARK
FORDHAM UNIVERSITY/MCGINLEY CENTER/RAMSKELLER KITCHEN
GREEN AND ACKERMAN KOSHER DAIRY RESTAURANT & PIZZA
HOMESTYLE FOOD SERVICES (ST. BARNABAS HIGH SCHOOL)
LOBBY LOUNGE AND TROUBLE'S TRUST @ THE PALACE HOTEL
NATURAL TOFU & NOODLES RESTAURANT (BOOK CHANG DONG)
NEW YORK BOTANICAL GARDENS TERRACE CAFE ( GARDEN CAFE )
NEW YORK UNIVERSITY - KIMMEL STUDENT CENTER CAFETERIA
PYRAMID COFFEE COMPANY HOSPITAL FOR SPECIAL SURGERY
Q.B.COMM.COLLEGE-MAIN KITCHEN/TIGER BITES PIZZA SECTION
RESORTS WORLD CASINO GROUND LEVEL ( EMPLOYEE DINING)
ROSE PIZZA, CHARLIE SUB GRILL, MOE'S SOUTHWEST GRILL
STARBUCKS (JFK TERMINAL 5-POST SECURITY

In the repetition operator {m,n}, we can skip putting the upper limit if we want to say, "anything with m matches and above". For example, let's find all the restaurants that have a name length 60 characters and above:

In [18]:
grep('^.{60,}$', 'data/uniquenames.txt')

(PUBLIC FARE) 81ST STREET AND CENTRAL PARK WEST (DELACORTE THEATRE)
BUFFALO WILD WINGS,PEETS COOFEE &TEA, PANOPOLIS BAKERY & CAFE
CENTER PLATE- CONCOURSE CAFE-JACOB K JAVITS CONVENTION CENTER
CENTERPLATE-EMPLOYEE CAFETERIA-JACOB K JAVITS CONVENTION CENTER
CENTRA`L MARKET ALL AMERICAN GRILL ( STATEN ISLAND FERRY TERMINAL)
DELTA SKY CLUB (BARTENDER SERVICE TERMINAL D DELTA DEPARTURE)
DUNKIN DONUTS (INSIDE GULF GAS STATION ON NORTH SIDE OF MAJ. DEEGAN EXWY- AFTER EXIT 13 - 233 ST.)
FASHION INSTITUTE OF TECHNOLOGY DAVID DUBINSKY STUDENT CENTER
GREATER NEW YORK SOCIAL AND HEALTH ADULT DAY CARE CENTER LLC
HOMEWOOD SUITES BY HILTON NEW YORK MIDTOWN MANHATTAN TIMES SQUARE
HONG KONG CAFE / FRESH SANDWICH BAKERY (BASEMENT FOOD COURT RESTAURANT & 1ST FL BAKERY)
MARLIN BAR AT TOMMY BAHAMA AND TOMMY BAHAMA RESTAURANT AND B
NEW WAI LING CHINESE RESTAURANT/NEW FRESCO TORTILLAS II TACO
NEW YORK UNIVERSITY - KIMMEL STUDENT CENTER (CATERING KITCHEN)
RED STORM DINER @ ST. VINCENT'S HALL OF ST. JOHN'S UNI

##### Repetition shortcuts (very common!): 

* `* = {0,}`. The `*` character means match the previous atom zero or more times
* `+ = {1,}`. The `+` character means match the previous atom one or more times
* `? = {0,1}`. The `*` character means match the previous atom zero or one times






Find all restaurants that start with one or more digits, followed by a space.

In [19]:
grep('^[0-9]+ ', 'data/uniquenames.txt')

002 MERCURY TACOS LLC
1 2 3 BURGER SHOT BEER
1 BANANA QUEEN
1 BUEN SABOR
1 DARBAR
1 EAST 66TH STREET KITCHEN
1 OAK
1 OR 8
1 STOP PATTY SHOP
10 DEVOE
10 POINTS KTV
100 FUN
1001 NIGHTS
1001 NIGHTS CAFE
1005 CATERING
101 CAFE
101 DELI
101 RESTAURANT AND BAR
102 NOODLES TOWN RESTAURANT
1020 BAR
1028 BAR & RESTAURANT EL SALVADORENO 
1061 CATERING LLC
107 WEST RESTAURANT
108 FAST FOOD CORP
108 LOUNGE - CLUB 108
1081 FULTON
11 STREET CAFE
111 RESTAURANT
1174 FULTON CUISINE, HALAL FOOD
12 CHAIRS
12 CHAIRS CAFE
12 CORAZONES RESTAURANT & BAR
12 CORNERS
12 CORNERS COFFEE INC
12 STREET ALE HOUSE
120 BAY CAFE
1200 MILES
121 FULTON STREET
123 NIKKO
124 COFFEE SHOP
128 DUMPLING HOUSE
137 BAR & GRILL
1380 BRONX RIVER CAFE CORP
149 STEAM FISH CORP.
15 CENTRAL PARK WEST RESTAURANT
15 EAST RESTAURANT
15 FLAVORS
15 FLAVORS 
15 ST CAFE
156 TEX BAR AND LOUNGE
16 HANDLES
16 HANDLES FROZEN YOGURT
16 HANDLES MIDWOOD
162 EB CORP BAKERY
168 ASUKA SUSHI
168 BOWERY HOLDING LLC
168 HI TEA
168 TEA LLC
169 BAR
1734 V

Find all restaurants that start with a letter, followed by one or more digits, followed by a space.

In [20]:
grep('^[A-Z][0-9]+ ', 'data/uniquenames.txt')

A1 OCHA SUSHI
B4 NYC
B66 CLUB
C2 CAFE
F1 LOUNGE AND GRILL
H20 LOUNGE AND RESTAURANT
J4 HOOKAH LOUNGE
Q2 THAI RESTAURANT
T2 - GO
T49 CAFE


Find all restaurants that start with the word STARBUCKS, followed by any number of characters, and then have a digit.

In [21]:
grep('STARBUCKS.*[0-9]+', 'data/uniquenames.txt')

STARBUCKS # 14840
STARBUCKS (JFK TERMINAL 5-POST SECURITY DEPARTURE)
STARBUCKS (STORE 16628)
STARBUCKS 22420
STARBUCKS COFFEE  #16608
STARBUCKS COFFEE # 15440
STARBUCKS COFFEE # 7463
STARBUCKS COFFEE # 7540
STARBUCKS COFFEE #14240
STARBUCKS COFFEE #18509
STARBUCKS COFFEE #20679
STARBUCKS COFFEE #21514
STARBUCKS COFFEE #22596
STARBUCKS COFFEE #23266
STARBUCKS COFFEE #23267
STARBUCKS COFFEE #3438
STARBUCKS COFFEE #7344
STARBUCKS COFFEE #7358
STARBUCKS COFFEE #7416
STARBUCKS COFFEE #7682
STARBUCKS COFFEE #7826
STARBUCKS COFFEE #9282
STARBUCKS COFFEE #9722
STARBUCKS COFFEE (#19890)
STARBUCKS COFFEE (#2785)
STARBUCKS COFFEE (STORE #13539)
STARBUCKS COFFEE (STORE #7216)
STARBUCKS COFFEE (STORE #7555)
STARBUCKS COFFEE (STORE #7577)
STARBUCKS COFFEE (STORE #7587)
STARBUCKS COFFEE (STORE #7696)
STARBUCKS COFFEE (STORE #8614)
STARBUCKS COFFEE (STORE 17478)
STARBUCKS COFFEE (STORE 7261)
STARBUCKS COFFEE (STORE#11650)
STARBUCKS COFFEE (STORE#20161)
STARBUCKS COFFEE COMPANY #22560
STARBUCKS COFFEE 

#### Grouping ()

In the group operator, when a group of characters is enclosed in parentheses, the next operator applies to the whole group, not only the previous characters. For example, find all restaurant names that contain BA two times or more:

In [22]:
grep(r'(BA){2}', 'data/uniquenames.txt')

ALI BABA
ALI BABA RESTAURANT
ALI BABA'S
ALIBABA
ALIBABA EXPRESS
ALIBABA GRILL
BABA COOL
BABA GHANOUGE
BABA'S PIEROGIES
BABAGHANOUSH
BABALU
SABABA LOUNGE


#### In class exercices

What do these regular expressions match?

* b (cd)*
* h (d)+
* j? k+
* (cd){2,5}
* o(pre){3,}
* Panos|Ipeirotis

#### In class exercises (advanced)

Write down the regular expressions for the following:

* A telephone number (e.g, 212-555-0921)
* A zip+4 code (e.g, 10012-1809)
* For matching a float number (e.g., +12.34 or -1.457 or 1023.4568)
* Dollar amount with optional cents  (e.g. \$0.33, \$784)
* Time of Day (e.g. 12:15am, 3:34pm)
* Match urls  only of the form http://www.alphanumeric.com
* Match an email of the form username@domain (assume  that the domain might be in the form alphanumeric.alphanumeric, or alphanumeric.alphanumeric.alphanumeric)   



### Backreferences

Sometimes it is handy to be able to refer to a match that was made earlier in a regex. This is done with backreferences. `\k` is the backreference specifier, where `k` is a number, which refers to the `k`-th regular expression *that was enclosed in parenthesis*.

For example, find if the first character(s) of a line are the same as the last:


In [23]:
grep(r'^(.{3,}).*\1$', 'data/uniquenames.txt')

108 LOUNGE - CLUB 108
ANTEK RESTAURANT
ANTOJITOS RETAURANT
ANTONIO'S RESTAURANT
ARRIBA ARRIBA
BARCELONA BAR
BARRACUDA BAR
BERONBERON
BINGO BINGO BINGO
BUMBLE AND BUMBLE
BURGER BURGER
CENTER PLATE- CONCOURSE CAFE-JACOB K JAVITS CONVENTION CENTER
CENTERPLATE-EMPLOYEE CAFETERIA-JACOB K JAVITS CONVENTION CENTER
CHARLES SALLY & CHARLES
CHEEBURGER CHEEBURGER
CHEN MOMMY KITCHEN
CHEN'S KITCHEN
CHOP CHOP
CREPE SUCRE
DIP DIP
ETCETERA ETCETERA
GAJI GAJI
GIT-IT-N-GIT
GONZALEZ Y GONZALEZ
GUDE GUDE
HALF AND HALF
HOME SWEET HOME
JANCHI JANCHI
KENEDY FRIED CHICKEN
KENNDY FRIED CHICKEN
KENNEDY  FRIED CHICKEN
KENNEDY FRIED CHICKEN
KENNEDY GRILL & FRIED CHICKEN
KENNEDY GRILL AND FRIED CHICKEN
KENNEDY PIZZA & CHICKEN
KENTUCKY FRIED CHICKEN
KOPI KOPI
LAS RAMBLAS
LES HALLES
LOS 3 POTRILLOS
LOS DOS POTRILLOS
LOS TRES POTRILLOS
LUCA LUCA
MANGO MANGO
MANGOMANGO
MOCU-MOCU
NAKA NAKA
NAM NAM
NITA NITA
NORTHERN MANOR
ONE AND ONE
PIO PIO
PIO-PIO
SAATCHI & SAATCHI
SALSA Y SALSA
SAN SOO KAP SAN
SANSOOKAPSAN
SEBA SEBA

Or find all the restaurant names that the first 5 characters (or more) are identical to the last characters.

In [24]:
grep(r'^([A-Z]+)\1$', 'data/uniquenames.txt')

BERONBERON
COCO
ISIS
MANGOMANGO
NONO
VIVI


Find all names that have three consecutive same digits

In [25]:
grep(r'([0-9])\1\1', 'data/uniquenames.txt')

111 RESTAURANT
444 MADISON COFFEE SHOP
555 VIVACAFE
777 THEATER BAR
888 KITCHEN
CAFE 2000 CORONA
CHEN BROTHERS 888 RESTAURANT, INC.
GALLAGHER'S 2000
LEGENDS 000
MEXICO 2000 DELI RESTAURANT
NEW 888 CHINA EXPRESS
OUTBACK STEAKHOUSE 3330
OUTBACK STEAKHOUSE 3332
STARBUCKS COFFEE (STORE #7555)
SUBWAY STORE 46555
TULCINGO DELI 111


As we are going to see, these backreferences will also be of tremendous use for extraction purposes.

#### In class exercise (advanced)

Say that you have a file with telephone numbers written in a variety of forms: 

* 679-397-5255
* 2126660921
* 212-998-0902
* 888-888-2222
* 800-555-1211
* 800 555 1212
* 800.555.1213
* (800) 555-1214
* 1-800-555-1215
* 1(800)555-1216
* 800-555-1212-1234
* 800-555-1212x1234
* 800-555-1212 ext. 1234
* work 1-(800) 555.1212 #1234

The task is to standardize everything in the form (xxx)-xxx-xxx.


To make the process interactive, go to http://regex101.com/?#python, copy and paste the numbers above in the textarea called "Text String", and then try to write the regular expression above. (As a side note, the website provides excellent explanations about the meaning of the regular expression that you write down.) Remember to put the "g" character in the small textfield next to the regex: this has the same meaning as in sed, and it means "find globally" the regex, not just the first occurence.


If you manage to deal with that task, consider the case of also having international country calling codes (e.g., +1 for US, +44 for UK, +7 for Russia, +30 for Greece, +354 for Iceland etc), and also standardizing the extensions.

### Additional Regex Resources

* [Visual Regular Expression Tester](http://www.debuggex.com/?flavor=pcre)
* [Test Python Regular Expressions Online](http://www.pyregex.com/)
* [Regular Expressions 101](http://regex101.com/?)
* [Python's re Library Official Documentation](http://docs.python.org/2/library/re.html)
* [Regular expression reference at W3schools](http://www.w3schools.com/jsref/jsref_obj_regexp.asp)
* [Parsing phone numbers using Python and regular expressions](http://www.diveintopython.net/regular_expressions/phone_numbers.html)

### Additional Regular Expressions

While we have not used these before, they are commonly used shortcuts to simplify the construction of regular expressions:

* `\d`: matches the digits, 0-9.
* `\D`: matches anything but `\d`.
* `\w`: matches any alphanumeric character plus underscore: `[A-Za-z0-9_]`.
* `\W`: matches anything but `\w`.
* `\s`: matches any "whitespace" character (space, tab, newline, etc): `[ \t\n\r\f\v]`.
* `\S`: matches anything but `\s`.
* `\b`: matches the breaks between alphanumeric and non-alphanumeric characters (an empty string), the boundary between `\w` and `\W`. Useful for ensuring that what you match is actually a word.
* `\B`: matches anything but `\b`. Useful for ensuring your match is in the middle of a word.

And the ones below get a little bit more advanced:

* `*?`, `+?`: ordinarily, `*`, `+` and `?` are greedy, matching the longest possible string that satisfies the regular expression. Adding the `?` to any of these makes it non-greedy, instead matching the shortest possible expression. 
* `(?: )`: A non-capturing group. This works just as `()`, but doesn’t hold on to the matched contents.
* `(?<=x)`: Matches any string that is preceded by x (an arbitrary regular expression).
